In [178]:
from enum import Enum, IntEnum
from dataclasses import dataclass, field
from typing import Tuple, List, Union, Optional
import itertools

Pos = Tuple[int, int]

class CellType(Enum):
  WRAP = 0
  OPEN = 1
  WALL = 2

  @staticmethod
  def from_string(s: str) -> 'CellType':
    if s == ' ':
      return CellType.WRAP
    elif s == '.':
      return CellType.OPEN
    elif s == '#':
      return CellType.WALL

@dataclass
class Span:
  lo: Optional[int] = None
  hi: Optional[int] = None

  def add(self, n: int):
    if self.lo is None or n < self.lo:
      self.lo = n
    if self.hi is None or n > self.hi:
      self.hi = n

  def wrap(self, n: int) -> int:
    return self.lo + ((n - self.lo) % (self.hi - self.lo + 1))

  def __contains__(self, n: int) -> bool:
    return self.lo <= n <= self.hi
  
@dataclass
class Map:
  height: int
  width: int
  cells: set
  walls: set
  start: Pos
  row_spans: List[Span]
  col_spans: List[Span]

  def wrap_col(self, p: Pos) -> Pos:
    r, c = p
    return (r, self.col_spans[r].wrap(c))

  def wrap_row(self, p: Pos) -> Pos:
    r, c = p
    return (self.row_spans[c].wrap(r), c)
  
  def render(self, p: Pos) -> str:
    if p in self.cells:
      return '.'
    elif p in self.walls:
      return '#'
    else:
      return ' '
  
  def is_wall(self, p: Pos) -> bool:
    return p in self.walls

  @staticmethod
  def from_string(s: str) -> 'Map':
    height = 1
    width = 1
    cells = set()
    walls = set()
    start = None
    for ry, row in enumerate(s.splitlines()):
      for cx, cell in enumerate(row):
        height = max(height, ry + 1)
        width = max(width, cx + 1)
        pos = (ry, cx)
        cell = CellType.from_string(cell)
        if cell == CellType.OPEN:
          cells.add(pos)
          if start is None:
            start = pos
        elif cell == CellType.WALL:
          walls.add(pos)
    row_spans = [Span() for _ in range(width)]
    col_spans = [Span() for _ in range(height)]
    for ry, cx in itertools.chain(cells, walls):
      row_spans[cx].add(ry)
      col_spans[ry].add(cx)
    return Map(height, width, cells, walls, start, row_spans, col_spans)

class Facing(IntEnum):
  R = 0
  D = 1
  L = 2
  U = 3

  def turn(self, s: str) -> 'Facing':
    if s == 'R':
      return Facing((self.value + 1) % 4)
    elif s == 'L':
      return Facing((self.value - 1) % 4)
    
  def to_string(self) -> str:
    if self == Facing.U:
      return '^'
    elif self == Facing.R:
      return '>'
    elif self == Facing.D:
      return 'v'
    elif self == Facing.L:
      return '<'

@dataclass
class Mover:
  map: Map
  moves: List[Union[int, str]]
  path: List[Tuple[Facing, Pos]] = field(default_factory=list)
  facing: Facing = Facing.R
  pos: Tuple[int, int] = field(init=False)
  
  def __post_init__(self):
    self.pos = self.map.start
    # self.path.append((self.facing, self.pos))

  def has_moves(self) -> bool:
    return len(self.moves) > 0
  
  def step(self):
    if self.moves[0] in ('R', 'L'):
      # print('turning', self.facing, self.moves[0])
      self.facing = self.facing.turn(self.moves[0])
      self.moves = self.moves[1:]
      return

    # print('moving', self.facing, self.moves[0])
    r, c = self.pos
    next = None
    if self.facing == Facing.U:
      next = self.map.wrap_row((r - 1, c))
    elif self.facing == Facing.R:
      next = self.map.wrap_col((r, c + 1))
    elif self.facing == Facing.D:
      next = self.map.wrap_row((r + 1, c))
    else:
      next = self.map.wrap_col((r, c - 1))

    # Block movement if wall.
    if self.map.is_wall(next):
      self.moves[0] = 0
    else:
      self.path.append((self.facing, self.pos))
      self.pos = next
      self.moves[0] -= 1
      
    if self.moves[0] == 0:
      self.moves = self.moves[1:]

  def render_path(self) -> str:
    grid = [[self.map.render((r, c)) for c in range(self.map.width)]
            for r in range(self.map.height)]
    for f, (r, c) in self.path + [(self.facing, self.pos)]:
      grid[r][c] = f.to_string()
    return ('\n'.join(''.join(r) for r in grid))
  
  def score(self) -> int:
    r, c = self.pos
    return 1000 * (r+1) + 4 * (c+1) + self.facing.value
    
def parse_moves(s: str):
  moves = []
  i = 0
  for j, v in enumerate(s):
    if v in ('L', 'R'):
      moves.append(int(s[i:j]))
      moves.append(s[j])
      i = j + 1
  if i != len(s):
    moves.append(int(s[i:]))
  return moves

def parse_puzzle_input(s: str):
  map, moves = s.split('\n\n')
  map = Map.from_string(map)
  moves = parse_moves(moves)

  print(f'parsed grid of ({map.height} rows x {map.width} cols)')
  return map, moves
  

In [179]:
puzzle = """        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5"""

map, moves = parse_puzzle_input(puzzle)
mover = Mover(map, moves)

parsed grid of (12 rows x 16 cols)


In [180]:
moves

[10, 'R', 5, 'L', 5, 'R', 10, 'L', 4, 'R', 5, 'L', 5]

In [181]:
while mover.has_moves():
  mover.step()

In [182]:
print(mover.render_path())
print(mover.score())

        >>v#    
        .#v.    
        #.v.    
        ..v.    
...#...v..v#    
>>>v...>#.>>    
..#v...#....    
...>>>>v..#.    
        ...#....
        .....#..
        .#......
        ......#.
6032


In [183]:
with open('day22.txt') as puzzle:
  map, moves = parse_puzzle_input(puzzle.read())
  mover = Mover(map, moves)
  while mover.has_moves():
    mover.step()
  print('Part 1:', mover.score())

parsed grid of (200 rows x 150 cols)
Part 1: 196134


In [ ]:
simple = """"
 A
 BCD
EF
"""